<div style='color: #690027;' markdown="1">
    <h1>Diep neuraal netwerk</h1> 
</div>

<div style='color: #690027;' markdown="1">
    <h2>Trainen van het netwerk</h2> 
</div>

Een diep neuraal netwerk bestaat uit lagen die opeenvolgend de invoer van het netwerk op een andere manier zullen voorstellen. De voorstelling in de latere lagen zal meer informatie bevatten in verband met een specifiek probleem.

Het netwerk zal de transformatie naar deze voorstellingen leren door beschikbare data te verwerken. De data bestaat uit invoer voor het netwerk, ook wel <b>samples</b> genoemd, en de verwachte uitvoer of <b>target</b> horende bij deze invoer. Het netwerk zal voor deze samples een voorspelling maken en vergelijken met de targets, op basis van deze vergelijking worden de interne parameters (ook <b>gewichten</b> genoemd) van het netwerk aangepast om ervoor te zorgen dat de volgende voorspelling dichter zal aanleunen bij de targets. Als voorbeeld gaan we een simpel diep neuraal netwerk trainen om een onderscheid te maken tussen een afbeelding met een stoma en een afbeelding zonder stoma. Volgende afbeelding toont enkele samples en targets die we hiervoor gaan gebruiken.

<img src="./images/training_data.jpg"/>

De beschikbare data wordt het best opgedeeld in 3 sets:
<ul>
    <li><b>Training set</b>: Dit is de data die gebruikt wordt om het model te trainen. De gewichten worden aangepast op basis van deze data.</li>
    <li><b>validatie set</b>: Deze data wordt gebruikt om te kijken hoe goed het model presteert op data die het nog niet gezien heeft. Op basis van deze data wordt de netwerk architectuur (of andere zaken) aangepast.</li>
    <li><b>Test set</b>: Na het trainen met de training set en bijstaven van het netwerk aan de hand van de validatie set wordt het netwerk nog 1 keer geëvalueerd met de test set om de eindscore te berekenen.</li>
</ul>

Voer onderstaande codecel uit om van de methodes in deze notebook gebruik te kunnen maken.

In [ ]:
from scripts import diep_neuraal_netwerk

Vooraleer we ons netwerk gaan trainen moeten we het eerst bouwen, we doen dit laag per laag. De eerste lagen zijn <b>convolutionele lagen</b>, ze voeren een convolutie operatie uit om patronen te herkennen in de afbeelding. De eerste convolutionele laag heeft 32 feature maps, bij de opeenvolgende convolutionele lagen verdubbeld het aantal feature maps steeds.

De convolutionele lagen worden afgewisseld met <b>max-pooling lagen</b>, deze lagen gaan uit een gespecifieerd venster (bij ons 2 op 2) enkel de hoogste waarde overhouden. Dit zorgt ervoor dat de latere lagen informatie bevatten over een groter gedeelte van de invoer en zo meer patronen kunnen herkennen. De grootte van de feature maps wordt ook gehalveerd waardoor het netwerk minder gewichten bevat. Volgende afbeelding geeft een voorbeeld van de max-pooling operatie.

<img src="./images/max_pooling.jpg"/>

Voer de volgende codecel uit om het aantal convolutionele lagen van het model te kiezen:

In [ ]:
display(diep_neuraal_netwerk.conv_base)

De convolutionele lagen gaan patronen herkennen op verschillende plaatsen in de afbeelding. Daarna zal een <b>flatten</b> operatie de 3-dimensionale uitvoer (lengte, breedte, feature maps) omzetten in naar een 1-dimensionale. Hierdoor kan de uitvoer verwerkt worden door <b>fully connected</b> of <b>feedforward lagen</b>, deze zullen de effectieve classificatie (wel of geen stoma) uitvoeren. De feedforward lagen bevatten neuronen die het aantal gewichten tussen deze lagen zal bepalen. Hoe meer neuronen hoe meer details het netwerk kan leren van de training data (dit is niet altijd goed!). De laatste laag bestaat uit slechts 1 neuron en geeft een getal tussen 0 en 1 terug, dit is de voorspelling van het netwerk waarbij 0 staat voor geen stoma en 1 voor wel een stoma.

Voer volgende codecel uit om het aantal feedforward lagen en het aantal neuronen van de eerste feedforward laag te kiezen.

In [ ]:
display(diep_neuraal_netwerk.ff_layers)
display(diep_neuraal_netwerk.ff_input)

Een andere belangrijke keuze bij het trainen van een diep neuraal netwerk is de <b>optimizer</b> en bijhorende <b>learning rate</b>, deze bepalen op welke manier het netwerk leert. Voor dit voorbeeld is er gekozen om de <b>stochastic gradient descent (SGD)</b> optimizer te gebruiken omdat deze gemakkelijk uit te leggen is. SGD maakt gebruik van de afgeleide om een minimum te vinden voor de <b>loss functie</b>, dit is een functie die beschrijft hoe goed het netwerk presteert met de huidige gewichten. Hoe lager de loss functie hoe dichter de huidige uitvoer van het netwerk bij de gewenste uitvoer ligt. Door de afgeleide in een punt te berekenen weet je welke richting zorgt voor de grootste daling van de functie. Op deze manier wordt een lokaal minimum gevonden. De learning rate bepaald hoe groot de stap is om in dit minimum te geraken, een kleine learning rate zal ervoor zorgen dat het netwerk traag leert en met een te grote learning rate zal het netwerk het minimum niet vinden. Volgende afbeelding toont het gradient descent werkt met 2 dimensies, bij een diep neuraal netwerk zijn er enorm veel gewichten en dus ook enorm veel dimensies wat veel moeilijker voor te stellen is door een figuur.

<img src="./images/gradient_descent.jpg" width="500"/>

Voer volgende codecel uit om de learning rate van het netwerk te kiezen.

In [ ]:
display(diep_neuraal_netwerk.learning_rate)

Nu we ons model hebben gespecifieerd zou het handig zijn om hier een visueel beeld van te hebben. Voer volgende codecel uit om je gekozen netwerk te visualiseren. Ben je niet tervreden of wil je andere zaken uitproberen verander dan gerust de parameters en voer de codecel opnieuw uit.

In [ ]:
diep_neuraal_netwerk.toon_netwerk()

Het netwerk is nu klaar om te trainen. De optimizer zal de gewichten aanpassen door de training data te verwerken. Je kan kiezen hoeveel keer er over de volledige training data gegaan wordt, dit wordt het aantal <b>epochs</b> genoemd.

Voer volgende codecel uit om het aantal epochs te kiezen.

In [ ]:
display(diep_neuraal_netwerk.epochs)

<div style='color: #690027;' markdown="1">
    <h2>Resultaten</h2> 
</div>

Normaal gezien duurt het even om een netwerk te trainen, de netwerken die in dit voorbeeld gemaakt kunnen worden zijn echter op voorhand getraind en de resultaten zijn opgeslaan in een databank. Op deze manier kunnen we dus onmiddellijk kijken naar de prestaties van het gekozen netwerk. 

Er bestaan verschillende manieren om een netwerk te beoordelen. Enerzijds kan je kijken naar de waarde van de loss functie, anderzijds kan je ook kijken naar de <b>accuracy</b> of nauwkeurigheid van het netwerk, dit is het percentage van de samples waarvoor het netwerk de voorspelling juist heeft. Deze waarden kan je tijdens het trainen per epoch berekenen voor de training data en de validatie data. Na het trainen kan je deze waarden ook nog 1 maal berekenen op de test data om de finale beoordeling van het netwerk te bekomen.

Door volgende codecel uit te voeren zie je de training, validatie en test accuracy voor het netwerk na het te trainen voor een hiervoor gespecifieerd aantal epochs. De grafiek geeft per epoch de training en de validatie accuracy weer.

In [ ]:
diep_neuraal_netwerk.toon_accuracy()
diep_neuraal_netwerk.toon_accuracy_grafiek()

Vaak zal je zien dat de validation accuracy ongeveer gelijk blijft terwijl de training accuracy stijgt. Wanneer de afstand tussen de 2 groter wordt op de grafiek dan is het netwerk aan het <b>overfitten</b>. Dit wil zeggen dat het netwerk te veel details van de training data leert en dus niet meer goed generaliseerd op data dat het nog nooit gezien heeft. Overfitting is 1 van de grootste problemen van een diep neuraal netwerk, daarom zal een volgende notebook een aantal technieken uitleggen om overfitting tegen te gaan.

Als extra tonen we nog een aantal voorspellingen van het netwerk. De voorspelling is een getal tussen 0 en 1 dat aangeeft hoe zeker het netwerk is dat er een stoma gevonden werd. Er zijn opzettelijk moeilijke voorbeelden bij zodat het netwerk soms denkt dat een stoma geen stoma is (<b>false negative</b>) of dat geen stoma wel een stoma is (<b>false positive</b>). 

Voer onderstaande codecel uit om de voorspellingen te zien.

In [ ]:
diep_neuraal_netwerk.toon_afbeeldingen()